In [48]:
import pickle
import codecs
import string
import math

import pymongo
from bson.code import Code

import pprint
import pandas as pd

import nltk
import string

import matplotlib.pyplot as plt
%matplotlib inline

In [71]:
def top_words(raw, n=20):
    '''
    Take raw text and return top 20 words, ignoring "words" that are punctuation
    word_list: list of tokens
    n: number of words to return
    '''
    tokens = nltk.word_tokenize(raw)
    fdist = nltk.FreqDist(w.lower() for w in tokens if (w.lower() not in stopwords) and (w.lower() not in string.punctuation))
    return fdist.most_common(n)

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [10]:
stopwords = nltk.corpus.stopwords.words('english')

In [3]:
nltk.corpus.gutenberg.fileids()

[u'austen-emma.txt',
 u'austen-persuasion.txt',
 u'austen-sense.txt',
 u'bible-kjv.txt',
 u'blake-poems.txt',
 u'bryant-stories.txt',
 u'burgess-busterbrown.txt',
 u'carroll-alice.txt',
 u'chesterton-ball.txt',
 u'chesterton-brown.txt',
 u'chesterton-thursday.txt',
 u'edgeworth-parents.txt',
 u'melville-moby_dick.txt',
 u'milton-paradise.txt',
 u'shakespeare-caesar.txt',
 u'shakespeare-hamlet.txt',
 u'shakespeare-macbeth.txt',
 u'whitman-leaves.txt']

In [58]:
caesar_words = nltk.corpus.gutenberg.words('shakespeare-caesar.txt')

In [22]:
len(caesar_words)

25833

### Most common words

In [24]:
fdist = nltk.FreqDist(w.lower() for w in caesar_words)

In [25]:
fdist.most_common(20)

[(u',', 2204),
 (u'.', 1296),
 (u'and', 627),
 (u'the', 579),
 (u'i', 533),
 (u':', 499),
 (u'to', 446),
 (u'you', 391),
 (u"'", 384),
 (u'of', 354),
 (u'?', 296),
 (u'that', 289),
 (u'a', 267),
 (u'not', 257),
 (u'is', 253),
 (u'in', 225),
 (u'd', 215),
 (u'my', 209),
 (u'it', 198),
 (u'caesar', 190)]

In [42]:
caesar_raw = nltk.corpus.gutenberg.raw('shakespeare-caesar.txt')
top_words(caesar_raw, 20)

[(u"'d", 193),
 (u'caesar', 190),
 (u'brutus', 161),
 (u'bru', 153),
 (u'haue', 147),
 (u'shall', 125),
 (u'thou', 115),
 (u'cassi', 107),
 (u'cassius', 85),
 (u'antony', 75),
 (u'come', 74),
 (u'let', 71),
 (u'good', 70),
 (u'know', 68),
 (u'men', 64),
 (u'enter', 64),
 (u'heere', 59),
 (u'vs', 58),
 (u'thy', 56),
 (u'thee', 55)]

In [43]:
hamlet_raw = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')
top_words(hamlet_raw, 20)

[(u'ham', 337),
 (u'lord', 211),
 (u"'d", 200),
 (u'haue', 175),
 (u'king', 172),
 (u"'s", 119),
 (u'shall', 107),
 (u'come', 104),
 (u'thou', 104),
 (u'let', 104),
 (u'hamlet', 100),
 (u'good', 98),
 (u'hor', 95),
 (u'thy', 90),
 (u'enter', 85),
 (u'oh', 81),
 (u'like', 77),
 (u'well', 70),
 (u'know', 69),
 (u'would', 68)]

In [72]:
macbeth_raw = nltk.corpus.gutenberg.raw('shakespeare-macbeth.txt')
top_words(macbeth_raw, 20)

[(u"'d", 192),
 (u'macb', 137),
 (u"'s", 128),
 (u'haue', 122),
 (u'thou', 87),
 (u'enter', 81),
 (u'shall', 68),
 (u'macbeth', 62),
 (u'thee', 61),
 (u'vpon', 59),
 (u'macd', 58),
 (u'yet', 57),
 (u'thy', 56),
 (u'vs', 56),
 (u'come', 54),
 (u'king', 53),
 (u'hath', 52),
 (u'rosse', 49),
 (u'lady', 48),
 (u'good', 48)]

### Collocations

In [91]:
measures = nltk.collocations.BigramAssocMeasures
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(caesar_words)

In [92]:
measures_dct = {'chi_sq': measures.chi_sq, 
            'dice': measures.dice, 
            'fisher': measures.fisher,
            'jaccard': measures.jaccard, 
            'likelihood': measures.likelihood_ratio, 
            'mi_like': measures.mi_like,
            'phi_sq': measures.phi_sq, 
            'pmi': measures.pmi, 
            'poisson_sterling': measures.poisson_stirling,
            'raw_freq': measures.raw_freq}

In [94]:
for name, measure in measures_dct.items():
    print name
    pprint.pprint(bigram_finder.nbest(measure, 10))

phi_sq
[(u'Attendants', u'absent'),
 (u'Barren', u'touched'),
 (u'Captiue', u'bonds'),
 (u'Cauerne', u'darke'),
 (u'Chimney', u'tops'),
 (u'Ciuill', u'strife'),
 (u'Coffers', u'fill'),
 (u'Comets', u'seen'),
 (u'Cynicke', u'rime'),
 (u'Darts', u'inuenomed')]
mi_like
[(u"'", u'd'),
 (u',', u'and'),
 (u'Bru', u'.'),
 (u'Cassi', u'.'),
 (u',', u'And'),
 (u"'", u's'),
 (u'my', u'Lord'),
 (u'I', u'am'),
 (u"'", u'st'),
 (u'any', u'thing')]
raw_freq
[(u',', u'and'),
 (u"'", u'd'),
 (u'Bru', u'.'),
 (u',', u'And'),
 (u',', u'I'),
 (u'Cassi', u'.'),
 (u'.', u'I'),
 (u',', u'that'),
 (u'.', u'Enter'),
 (u'Brutus', u',')]
dice
[(u'Attendants', u'absent'),
 (u'Barren', u'touched'),
 (u'Captiue', u'bonds'),
 (u'Cauerne', u'darke'),
 (u'Chimney', u'tops'),
 (u'Ciuill', u'strife'),
 (u'Coffers', u'fill'),
 (u'Comets', u'seen'),
 (u'Cynicke', u'rime'),
 (u'Darts', u'inuenomed')]
chi_sq
[(u'Attendants', u'absent'),
 (u'Barren', u'touched'),
 (u'Captiue', u'bonds'),
 (u'Cauerne', u'darke'),
 (u'Chimney

In [100]:
for name, measure in measures_dct.items():
    print name
    pprint.pprint(bigram_finder.nbest(measure, 10))
    bigram_finder.apply_freq_filter(5)
    pprint.pprint(bigram_finder.nbest(measure, 10))

phi_sq
[(u"'", u'd'),
 (u'Caius', u'Ligarius'),
 (u'Metellus', u'Cymber'),
 (u'any', u'thing'),
 (u'Mark', u'Antony'),
 (u'my', u'Lord'),
 (u'Marke', u'Antony'),
 (u"'", u's'),
 (u'mine', u'owne'),
 (u'Bru', u'.')]
[(u"'", u'd'),
 (u'Caius', u'Ligarius'),
 (u'Metellus', u'Cymber'),
 (u'any', u'thing'),
 (u'Mark', u'Antony'),
 (u'my', u'Lord'),
 (u'Marke', u'Antony'),
 (u"'", u's'),
 (u'mine', u'owne'),
 (u'Bru', u'.')]
mi_like
[(u"'", u'd'),
 (u',', u'and'),
 (u'Bru', u'.'),
 (u'Cassi', u'.'),
 (u',', u'And'),
 (u"'", u's'),
 (u'my', u'Lord'),
 (u'I', u'am'),
 (u"'", u'st'),
 (u'any', u'thing')]
[(u"'", u'd'),
 (u',', u'and'),
 (u'Bru', u'.'),
 (u'Cassi', u'.'),
 (u',', u'And'),
 (u"'", u's'),
 (u'my', u'Lord'),
 (u'I', u'am'),
 (u"'", u'st'),
 (u'any', u'thing')]
raw_freq
[(u',', u'and'),
 (u"'", u'd'),
 (u'Bru', u'.'),
 (u',', u'And'),
 (u',', u'I'),
 (u'Cassi', u'.'),
 (u'.', u'I'),
 (u',', u'that'),
 (u'.', u'Enter'),
 (u'Brutus', u',')]
[(u',', u'and'),
 (u"'", u'd'),
 (u'Bru', u'

In [101]:
from textblob import TextBlob

In [102]:
caesar_blob = TextBlob(nltk.corpus.gutenberg.raw('shakespeare-caesar.txt'))

In [116]:
caesar_blob.sentiment

Sentiment(polarity=0.17182804312804303, subjectivity=0.5805136335136323)